In [78]:
# Project: Write a Data Science Blog Post
# James l. Sturges
# The following Code runs a Linear regression analysis on AirBnB data
# It also converts the gz file to a csv file for Tableau

##### The purpose of this project is to answer 3-5 questions about the data.

### The Three Question I chose were:
+ **Where are AirBnB properties distributed across Seattle and does the appear to affect price?**
      This question was answered via Tableau mapping feature and is documented in the Blog post and in the Tableau Twbx file include in the project sumission

+ **What features appear to impact the price the most?**

     This question was answered by the running the linear regression model and analysing the coefficent values, also noted in the cells below
     
     
+ **Does the cancellation policy have a significant impact on price?**

     This question was also answered by the running the linear regression model and analysing the coefficent values, also noted in the cells below


In [79]:
'''
Description: This code block does the initial housekeeping for the program including:
* Library imports
* Directory variable defined, used for data location
* columns to be used in data file
'''

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
import glob
import datetime
from datetime import datetime as dt

drct='C:/Users/Jim.000/Documents/Udacity Data Scientist Class/Project - Write a Data Science Blog Post/Data/393_43835_bundle_archive/S_201901_listings.csv'

#display (list_of_files)
# during analysis where high correlation was indicated (for example, between zip codes and neighborhoods 
# I eliminated the columns from this list)
column_list=[
     'host_response_time',
     'host_response_rate',
     'host_is_superhost',
     'host_listings_count',
     'zipcode',
     'property_type',
     'room_type',
     'accommodates',
     'bathrooms',
     'bedrooms',
     'beds',
     'bed_type',
     'square_feet',
     'price',
     'guests_included',
     'minimum_nights',
     'maximum_nights',
     'has_availability',
     'number_of_reviews',
     'number_of_reviews_ltm',
     'review_scores_rating',
     'review_scores_accuracy',
     'review_scores_value',
     'instant_bookable',
     'is_business_travel_ready',
     'cancellation_policy',
     'calculated_host_listings_count',
     'reviews_per_month']
x_var_list=['bathrooms','bedrooms','square_feet','guests_included','minimum_nights','maximum_nights']
y_var_list=['price']



In [80]:
def read_data():
    '''
    Description: This function reads the data in from the directory
                 It reads in a number of files (ext:gz) and concatinates the result before returning a data frame
    Arguments: None
    Returns: A csv fileis written out for use by Tableau and the resultant dataframe
    '''
    list_of_files = glob.glob(drct+'/*.gz')
    list_of_files=list_of_files[0:1] #only use first one for now
    for indx, file in enumerate(list_of_files):
        if indx==0:
            df=pd.read_csv(file, compression='gzip', header=0, sep=',', quotechar='"',usecols=column_list,low_memory=False)
        else:
            df=pd.concat([df,pd.read_csv(file, compression='gzip', header=0, sep=',', quotechar='"',usecols=column_list,low_memory=False)])
    df.to_csv(r'C:\Users\Jim.000\Desktop\airbnb.csv')
    return(df)

In [81]:
def clean_and_fill_numeric(df):
    '''
    Description: This function Converts data types as appropriate and fills na numerics with the mean
    Arguments: Dataframe of features (X)
    Returns: Modified Dataframe
    '''   
    
    # convert zipcode to string
    df['zipcode'] =  df['zipcode'].astype('str')
    df['zipcode'] =  df['zipcode'].str.zfill(5)

    #convert price to numeric and drop any missing
    df['price']=[x[1:] for x in df.price]
    df['price']= df['price'].str.replace(',', '')
    df['price']=pd.to_numeric(df['price'])
    df=df.dropna(subset=['price'],axis=0)

    #convert host reponse to numeric
    df['host_response_rate']=pd.to_numeric(df['host_response_rate'].str.strip('%'))/100

    #Fill missing numerics with mean
    num_vars=df.select_dtypes(include=['float64','int64']).columns
    for col in num_vars:
        df[col].fillna((df[col].mean()), inplace=True) #fill na with the mean value
    return df

In [82]:
def create_dummy_df(df,cat_cols):
    '''
    Description: Routine creates dummy variable for the dataframe
    Arguments: Dataframe, catigorical column list, for the catigorical columns passed in and returns the dataframe
    Returns: Modified Dataframe
    '''
    for col in cat_cols:
        df = pd.concat([df.drop(col,axis=1), \
                pd.get_dummies(df[col],prefix=col,prefix_sep='_',
                              drop_first=True)],axis=1)
    return df

In [83]:
def find_optimal_lm_mod(X,y,cutoffs,test_size=.30, random_state=42):
    '''
    Description: This routine determines the optimal features to include in a linear regression model 
                    but applying different cutoff values for the number of missing values for a given feature.
    Arguments: 
        * The response Variable Series (y)
        * The feature set X
        * A set of cuttoff points to eliminated features where 
          the number of values with in a given feature were not adequate
        * A test sample size and random state (for repeatable results)
        
    Returns: The output is the optimal linear regression model

    Ref credit: This routine used in Udacity training with additional personal notes included
    '''
    r2_scores_test,r2_scores_train, num_feats, results = [], [],[], dict()
    for cutoff in cutoffs:
        reduce_X = X.iloc[:, np.where((X.sum() > cutoff) == True)[0]] #proportion of features that are 1
        num_feats.append(reduce_X.shape[1])
       
        #split into train and test
        X_train,X_test,y_train, y_test= train_test_split(reduce_X,y,test_size=.30,random_state=42)
        
        #instantiate and fit
        lm_model = LinearRegression(normalize=True) 
        lm_model.fit(X_train,y_train)
        y_test_preds=lm_model.predict(X_test)
        y_train_preds=lm_model.predict(X_train)
        #display("r-squared for the model was {} at a cutoff of {} missing values in a column on {} features." \
            #.format(r2_score(y_test,y_test_preds),cutoff, len(X_test.columns)))
        
        #append the r2 value for the test set
        r2_scores_test.append(r2_score(y_test,y_test_preds))
        r2_scores_train.append(r2_score(y_train,y_train_preds))
        results[str(cutoff)]=r2_score(y_test,y_test_preds)
        
    best_cutoff=max(results, key=results.get)
    #print("best cutoff was:", best_cutoff)
    print(best_cutoff, results[best_cutoff])
    display("r-squared for the best model was {} at a cutoff of {} missing values in a column." \
            .format(results[best_cutoff],best_cutoff))
     
    # Now that best cutoff is estblished run the regression model a last time with that cuttoff
    #reduce X matrix
    reduce_X = X.iloc[:, np.where((X.sum() > int(best_cutoff)) == True)[0]]
    num_feats.append(reduce_X.shape[1])
    
    #split into train and test
    X_train,X_test,y_train, y_test= train_test_split(reduce_X,y,test_size=.30,random_state=42)
    
    #instantiate and fit
    lm_model = LinearRegression(normalize=True) 
    lm_model.fit(X_train,y_train)
    
    return r2_scores_test, r2_scores_train, lm_model, X_train, X_test, y_train, y_test




In [84]:
def coef_weights(lm_model,X_train):
    '''
    Description: Pulls the coefficent weights from a linear model a creates a dataframe of values
    Arguments: lenear model, feature dataframe
    Returns: dataframe of coefficents and absolute value of coefficents (sorted decending) 
        
    Ref credit: This routine used in Udacity training with additional comments added
    '''
    coefs_df = pd.DataFrame()
    coefs_df['est_int']=X_train.columns
    coefs_df['coefs']=lm_model.coef_
    coefs_df['abs_coefs']=np.abs(lm_model.coef_)
    coefs_df=coefs_df.sort_values('abs_coefs', ascending=False)
    return coefs_df

In [85]:
### Main Program ###

# read in data
df=read_data()
#limit analysis to full structures
df=df[df['room_type']=='Entire home/apt'].copy()
       
# clean and fill numerics with mean
df = clean_and_fill_numeric(df)

# dummy the categorical variables
cat_vars=df.select_dtypes(include=['object']).columns
df=create_dummy_df(df,cat_vars)

#find the optimal fit using cutoffs
#Where cutoffs here pertains to the number of missing values allowed in the used columns.
#Therefore, lower values for the cutoff provides more predictors in the model.
cutoffs = [2500, 1000, 100, 50, 30, 25, 15, 10, 3]
X=df.drop(['price'], axis=1)
y=df['price']
r2_scores_test, r2_scores_train, lm_model, X_train, X_test, y_train, y_test=find_optimal_lm_mod(X,y,cutoffs)

#Examine heat map for correlated features
# I used this corrilation to remove highly correlated features
#plt.figure(figsize = (10, 8))
#sns.heatmap(X.corr(), cmap="RdYlGn")
#plt.suptitle("Pearson Correlation Heatmap")
#plt.show()



        

100 0.541876244012433


'r-squared for the best model was 0.541876244012433 at a cutoff of 100 missing values in a column.'

Now that we have run the linear regression we are ready to analyze the coeffiecient values.


## Analysis for Questions 2 & 3

### Having completed the linear regression we can examine the Coeffiecient weights to answer Questions 2 and 3


+ **What features appear to impact the price the most?**

      We see that the top features affecting price are cancellation, locations (zip code) and size (bedrooms and bathrooms)
     
     
+ **Does the cancellation policy have a significant impact on price?**

      Yes, cancellation policy shows up twice in the top 5 coeffiecients

In [86]:
# Examine the coefficient weights
coef_weights(lm_model,X_train).head(20)

,est_int,coefs,abs_coefs
48,cancellation_policy_strict,-112.931503,112.931503
34,zipcode_98121.0,106.998632,106.998632
21,zipcode_98101.0,105.485085,105.485085
33,zipcode_98119.0,99.164771,99.164771
50,cancellation_policy_super_strict_30,-84.125268,84.125268
39,property_type_Condominium,-76.063509,76.063509
24,zipcode_98104.0,66.383702,66.383702
27,zipcode_98109.0,66.087300,66.087300
28,zipcode_98112.0,62.124486,62.124486
22,zipcode_98102.0,57.611243,57.611243
